In [1]:
import pathlib
from Utils.wetland_selection    import get_top_scenes
from Utils.treatment_regression import logistic_process
from Utils.outcome_syndata_generator import generate_synthetic_outcome
from Utils.treatment_syndata_generator import generate_synthetic_treatment
from Utils.outcome_regression import regression_process
from Utils.convolution import make_exp_kernel
from Utils.outcome_post import generate_post
import numpy as np

In [2]:
# top scenes by wetland development
top_df = get_top_scenes(num_scenes=5)
scene_ids = top_df['scene'].astype(str).tolist()

# define folder paths for rasters
DATA_DIR = pathlib.Path('/Users/sayedmorteza/Library/CloudStorage/Box-Box/Hetwet_Data')         

# folders
folders = {
    'wet': DATA_DIR / 'WETLAND_DEV_1996_2016',
    'dem': DATA_DIR / 'DEM',
    'cap': DATA_DIR / 'CAPITAL_1996',
    'claims_96': DATA_DIR / 'LOG_CLAIMS_1996',
    'claims_16': DATA_DIR / 'LOG_CLAIMS_2016',

}

In [3]:
# estimate treatment logistic regression parameters
theta = 0.5  # classification threshold
logit_res = logistic_process(
    scene_ids=scene_ids,
    folders=folders,
    threshold=theta,
    regularization_C=1.0,
    noise_sd=0.1,
    noise_type='gaussian'
)

INFO: Processing scene 3043
INFO: Processing scene 2985
INFO: Processing scene 3095
INFO: Processing scene 2779
INFO: Processing scene 2922
INFO: Coefficients saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/treatment_logistic_coeffs.txt
INFO: Maps saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/scene_3043_treatment_maps.pdf


In [4]:
# generate synthetic treatment
generate_synthetic_treatment(
    scene_ids=scene_ids,
    folders=folders,
    logit_pipe=logit_res,
    threshold=theta,
    noise_sd=0.1,
    noise_type='gaussian'
)


INFO: [gaussian] Saved synthetic for 3043
INFO: [gaussian] Saved synthetic for 2985
INFO: [gaussian] Saved synthetic for 3095
INFO: [gaussian] Saved synthetic for 2779
INFO: [gaussian] Saved synthetic for 2922
INFO: All outputs written to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results


In [5]:
# estimate outcome linear regression parameters
outcome_reg = regression_process(
    scene_ids=scene_ids,
    folders=folders,
    noise_sd=0.1,
    noise_type='gaussian'
)

INFO: Loading scene 3043
INFO: Loading scene 2985
INFO: Loading scene 3095
INFO: Loading scene 2779
INFO: Loading scene 2922
INFO: Coefficients saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/outcome_regression_coeffs.txt
INFO: Maps saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/scene_3043_continuous_outcome.pdf


In [6]:
# generate synthetic outcomes:
generate_synthetic_outcome(
    scene_ids = scene_ids,
    folders = folders,
    reg_pipe = outcome_reg,
    target_shape = (8, 8),
    noise_type = "gaussian",
    noise_sd = 0.2,
    results_dir = None
)

INFO: [3043] Saved synthetic → scene_3043_synthetic_gaussian.tiff
INFO: [3043] Saved comparison PDF → 3043_comparison_gaussian.pdf
INFO: [2985] Saved synthetic → scene_2985_synthetic_gaussian.tiff
INFO: [2985] Saved comparison PDF → 2985_comparison_gaussian.pdf
INFO: [3095] Saved synthetic → scene_3095_synthetic_gaussian.tiff
INFO: [3095] Saved comparison PDF → 3095_comparison_gaussian.pdf
INFO: [2779] Saved synthetic → scene_2779_synthetic_gaussian.tiff
INFO: [2779] Saved comparison PDF → 2779_comparison_gaussian.pdf
INFO: [2922] Saved synthetic → scene_2922_synthetic_gaussian.tiff
INFO: [2922] Saved comparison PDF → 2922_comparison_gaussian.pdf
INFO: All outcome TIFFs and PDFs written to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/Outcome


In [7]:
# build your spillover kernel
CELL_SIZE = 960 # cell size in meters
LAMBDA    = 5000.0
KERNEL    = make_exp_kernel(lam_m=LAMBDA, cell_size_m=CELL_SIZE, truncate=4)

In [8]:
generate_post(
    scene_ids=scene_ids,
    folders=folders,
    KERNEL=KERNEL,
    Bbase = 0.02,
    Beta1 = 0.1,
    Beta2 = 1
)

/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
INFO: [3043] Saved post outcome coarse → scene_3043_post_gaussian.tiff
INFO: [3043] Saved ITE coarse → scene_3043_ITE.tiff
INFO: [3043] Saved comparison PDF → 3043_comparison_gaussian.pdf
/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
INFO: [2985] Saved post outcome coarse → scene_2985_post_gaussian.tiff
INFO: [2985] Saved ITE coarse → scene_2985_ITE.tiff
INFO: [2985] Saved comparison PDF → 2985_comparison_gaussian.pdf
/opt/miniconda3/envs/imageTRT/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeor